In [1]:
from pathlib import Path

import cv2
import numpy as np
import imageio.v3 as iio

import ipywidgets as widgets
from matplotlib import pyplot as plt
from IPython.display import display, Image, Video

from ipywebrtc import CameraStream, ImageRecorder, VideoRecorder
camera = CameraStream(constraints={
    "facing_mode": "user",
    "audio": False,
    "video": { "width": 640, "height": 480 }

})


recorder = ImageRecorder(stream=camera)

stop_button = widgets.Button(description="Stop Camera", icon="stop", button_style="danger")
stop_button.on_click(lambda _: camera.close())

widgets.VBox([ widgets.HBox([ camera, recorder ]), stop_button ])



In [28]:
#predict
from LE.model_no_cord import UNetModelFlatten
import torch
import dsntnn
snapshot = iio.imread(recorder.image.value, mode="RGB")


snap=torch.tensor(snapshot)
snap =torch.reshape(snap,(3,640,480))
print(type(snap[0][0][0]))

model = UNetModelFlatten(4, 3, 20, 64)
model.load_state_dict(torch.load("./flatten4.pth",map_location=torch.device('cpu')))
model.eval()

prediction= model(snap)
unnomilizedpred=dsntnn.normalized_to_pixel_coordinates(prediction,(480,640))





<class 'torch.Tensor'>


RuntimeError: expected scalar type Byte but found Float

In [ ]:
import numpy as np
import math
import pickle
import matplotlib.pyplot as plt
#from LE.testUnet import UNetModel
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split


numpy_data = unnomilizedpred.numpy()
distance_matrix = np.zeros((20,20))

for i in range(len(numpy_data)):
    #Check that the joint is visible
    if(numpy_data[i][0] != 0 and numpy_data[i][1] != 0):
        #For each other joint in the frame
        for j in range(len(numpy_data)):
            #Check that the joint is visible
            if(numpy_data[i][0] != 0 and numpy_data[i][0] != 0):
                #Calculate distance between joint i and all joints j and add to matrix
                distance_matrix[i,j] = math.sqrt((numpy_data[i][0] - numpy_data[j][0])**2 + (numpy_data[i][1] - numpy_data[j][1])**2)

#Flatten list 
flattened_distances = np.reshape(distance_matrix, (1,400))


#Get the actual gesture based on prediction number
gestures = ["ASL_letter_A", "ASL_letter_B", "ASL_letter_C", "ASL_letter_L", "ASL_letter_R", "ASL_letter_U",
"ASL_letter_G", "ASL_letter_H", "ASL_letter_I", "ASL_letter_V", "ASL_letter_W", "ASL_letter_Y"]

# Load model
file = open('finalized_linear_model', 'rb')
GR_model = pickle.load(file)
#print(GR_model.config)

pred = GR_model.predict(flattened_distances)

print("The predicted gesture from input data is:", gestures[int(pred[0])])